# Transport for London (TfL) Journeys - Exploratory Data Analysis

This is an exploratory data analysis project. The aim of this project is to identify the main characteristics of a dataset, extract information without using a formal model and display insights visually in a way that would be understood by stakeholders.

I chose this dataset to analyse, because I am a regular user of the Transport for London (TfL) network. I find myself constantly trying to avoid travelling during busier times or at busier stations. I thought that it would be interesting to analyse the data from the point of view of a TfL employee who is trying to reduce overcrowded stations.

## The Dataset

The dataset contains data on Oyster Card journeys from a week in November of 2009. Oyster cards are reusable cards used for ticketing purposes on the TfL network, so they can be used on buses, underground trains, tram lines etc.

The features in the dataset are either about the journey taken (e.g. Start/Exit station, Route numbers) or the Oyster Card type. 

### Limitations of the Dataset 

- All the data is from a single week in November 2009, that may not be representative of all weeks.
- Since the data is from over 15 years ago, it may inaccurately describe what journeys would look like today. Many things have changed such as new service lines, different payment methods. Because of this, I want to ensure that any analysis applied on this dataset could be applied on a dataset from 2024 as well.

## Analysis

Questions to answer in the initial analysis:
- How many journeys were made for each mode of transport (i.e. bus, DLR, underground etc.)?
- What is the distribution of journey start times? This can help to identify the busiest times.
- Which stations are used the most?
- Which products are used to pay for journeys the most/least?